## Imports

In [1]:
import numpy as np
import utils
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


## Set hyperparameters

In [2]:
label_mode = 'fine'
seed = 42
np.random.seed(seed)
batch_size = 32
n_epochs = 100

save_dir = os.path.join(os.getcwd(), 'saved_models')

## Load Cifar100 dataset

In [3]:
x, y_c, y_f, xt, yt_c, yt_f = utils.load_dataset()

X_train = x.astype('float32')
X_test = xt.astype('float32')

if label_mode == 'fine':
    n_classes = 100
    y_train = y_f
    y_test = yt_f
else:
    n_classes = 20
    y_train = y_c
    y_test = yt_c

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

Dataset already downloaded. Did not download twice.
Dataset already extracted. Did not extract twice.
X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Create k fold for cross validation

In [4]:
X = np.concatenate([X_train, X_test], axis=0)
Y = np.concatenate([y_train, y_test], axis=0)

kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=seed)
cvscores = []

## Create and train model

In [5]:
for train, test in kfold.split(X, Y):
    # Create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    # Optimizer
    opt = optimizers.rmsprop(lr=0.0001, decay=1e-6)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    # Data augmentation
    datagen = ImageDataGenerator(
        rescale = 1./255,  
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip=True)
    
    X_train = X[train]
    X_test = X[test]

    datagen.fit(X_train)

    X_test = X_test/255.
  
    # Convert to one hot encoding
    Y_train = np.eye(n_classes)[Y[train]]
    Y_test = np.eye(n_classes)[Y[test]]

    # Train
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        steps_per_epoch=len(X[train])//batch_size + 1,
                        epochs=n_epochs,
                        validation_data=(X_test, Y_test),
                        verbose=2)
                    
    # Evaluate performance
    scores = model.evaluate(X_test, Y_test, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
print("Cross validation scores: ")
print(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))  

Epoch 1/100
 - 22s - loss: 4.3460 - acc: 0.0383 - val_loss: 4.1177 - val_acc: 0.0826
Epoch 2/100
 - 19s - loss: 3.9669 - acc: 0.0927 - val_loss: 3.6542 - val_acc: 0.1602
Epoch 3/100
 - 18s - loss: 3.7205 - acc: 0.1330 - val_loss: 3.4682 - val_acc: 0.1890
Epoch 4/100
 - 19s - loss: 3.5618 - acc: 0.1576 - val_loss: 3.2894 - val_acc: 0.2179
Epoch 5/100
 - 18s - loss: 3.4322 - acc: 0.1798 - val_loss: 3.1711 - val_acc: 0.2409
Epoch 6/100
 - 18s - loss: 3.3360 - acc: 0.1976 - val_loss: 3.1460 - val_acc: 0.2443
Epoch 7/100
 - 18s - loss: 3.2552 - acc: 0.2111 - val_loss: 2.9970 - val_acc: 0.2686
Epoch 8/100
 - 19s - loss: 3.1876 - acc: 0.2275 - val_loss: 2.9423 - val_acc: 0.2810
Epoch 9/100
 - 18s - loss: 3.1321 - acc: 0.2352 - val_loss: 2.8574 - val_acc: 0.2957
Epoch 10/100
 - 19s - loss: 3.0816 - acc: 0.2470 - val_loss: 2.8639 - val_acc: 0.2965
Epoch 11/100
 - 19s - loss: 3.0413 - acc: 0.2518 - val_loss: 2.8471 - val_acc: 0.2974
Epoch 12/100
 - 19s - loss: 2.9900 - acc: 0.2641 - val_loss: 2.

Epoch 97/100
 - 18s - loss: 2.4705 - acc: 0.3742 - val_loss: 2.2352 - val_acc: 0.4262
Epoch 98/100
 - 19s - loss: 2.4730 - acc: 0.3729 - val_loss: 2.2536 - val_acc: 0.4191
Epoch 99/100
 - 19s - loss: 2.4811 - acc: 0.3711 - val_loss: 2.3610 - val_acc: 0.3884
Epoch 100/100
 - 18s - loss: 2.4730 - acc: 0.3721 - val_loss: 2.4140 - val_acc: 0.4042
acc: 40.42%
Epoch 1/100
 - 19s - loss: 4.3338 - acc: 0.0420 - val_loss: 3.9844 - val_acc: 0.1036
Epoch 2/100
 - 18s - loss: 3.9522 - acc: 0.0935 - val_loss: 3.6695 - val_acc: 0.1547
Epoch 3/100
 - 18s - loss: 3.7222 - acc: 0.1331 - val_loss: 3.4615 - val_acc: 0.1789
Epoch 4/100
 - 19s - loss: 3.5503 - acc: 0.1600 - val_loss: 3.2822 - val_acc: 0.2178
Epoch 5/100
 - 18s - loss: 3.4163 - acc: 0.1840 - val_loss: 3.1722 - val_acc: 0.2329
Epoch 6/100
 - 18s - loss: 3.3117 - acc: 0.2029 - val_loss: 3.0511 - val_acc: 0.2567
Epoch 7/100
 - 18s - loss: 3.2330 - acc: 0.2181 - val_loss: 2.9708 - val_acc: 0.2715
Epoch 8/100
 - 18s - loss: 3.1625 - acc: 0.2290 

Epoch 93/100
 - 22s - loss: 2.4213 - acc: 0.3826 - val_loss: 2.3272 - val_acc: 0.4080
Epoch 94/100
 - 22s - loss: 2.4338 - acc: 0.3816 - val_loss: 2.3374 - val_acc: 0.4117
Epoch 95/100
 - 22s - loss: 2.4352 - acc: 0.3801 - val_loss: 2.2557 - val_acc: 0.4242
Epoch 96/100
 - 22s - loss: 2.4255 - acc: 0.3842 - val_loss: 2.2433 - val_acc: 0.4277
Epoch 97/100
 - 22s - loss: 2.4360 - acc: 0.3795 - val_loss: 2.2507 - val_acc: 0.4244
Epoch 98/100
 - 22s - loss: 2.4318 - acc: 0.3804 - val_loss: 2.1874 - val_acc: 0.4290
Epoch 99/100
 - 21s - loss: 2.4320 - acc: 0.3827 - val_loss: 2.2947 - val_acc: 0.4065
Epoch 100/100
 - 21s - loss: 2.4338 - acc: 0.3841 - val_loss: 2.2216 - val_acc: 0.4247
acc: 42.47%
Epoch 1/100
 - 22s - loss: 4.3256 - acc: 0.0411 - val_loss: 3.9858 - val_acc: 0.1061
Epoch 2/100
 - 22s - loss: 3.9412 - acc: 0.0974 - val_loss: 3.6318 - val_acc: 0.1631
Epoch 3/100
 - 22s - loss: 3.7026 - acc: 0.1332 - val_loss: 3.4514 - val_acc: 0.1866
Epoch 4/100
 - 22s - loss: 3.5571 - acc: 0.1

Epoch 89/100
 - 18s - loss: 2.4742 - acc: 0.3741 - val_loss: 2.2097 - val_acc: 0.4326
Epoch 90/100
 - 18s - loss: 2.4556 - acc: 0.3768 - val_loss: 2.2006 - val_acc: 0.4312
Epoch 91/100
 - 19s - loss: 2.4780 - acc: 0.3726 - val_loss: 2.1927 - val_acc: 0.4411
Epoch 92/100
 - 19s - loss: 2.4724 - acc: 0.3757 - val_loss: 2.1508 - val_acc: 0.4436
Epoch 93/100
 - 18s - loss: 2.4671 - acc: 0.3736 - val_loss: 2.1523 - val_acc: 0.4482
Epoch 94/100
 - 18s - loss: 2.4730 - acc: 0.3737 - val_loss: 2.2138 - val_acc: 0.4241
Epoch 95/100
 - 19s - loss: 2.4762 - acc: 0.3745 - val_loss: 2.2883 - val_acc: 0.4111
Epoch 96/100
 - 19s - loss: 2.4677 - acc: 0.3738 - val_loss: 2.2904 - val_acc: 0.4116
Epoch 97/100
 - 19s - loss: 2.4715 - acc: 0.3736 - val_loss: 2.2574 - val_acc: 0.4164
Epoch 98/100
 - 19s - loss: 2.4749 - acc: 0.3725 - val_loss: 2.2193 - val_acc: 0.4302
Epoch 99/100
 - 19s - loss: 2.4690 - acc: 0.3753 - val_loss: 2.2545 - val_acc: 0.4230
Epoch 100/100
 - 18s - loss: 2.4707 - acc: 0.3746 - va

Epoch 85/100
 - 19s - loss: 2.4977 - acc: 0.3691 - val_loss: 2.3144 - val_acc: 0.4049
Epoch 86/100
 - 19s - loss: 2.5000 - acc: 0.3659 - val_loss: 2.3907 - val_acc: 0.3915
Epoch 87/100
 - 18s - loss: 2.4923 - acc: 0.3685 - val_loss: 2.4135 - val_acc: 0.4071
Epoch 88/100
 - 19s - loss: 2.4907 - acc: 0.3684 - val_loss: 2.3092 - val_acc: 0.4166
Epoch 89/100
 - 18s - loss: 2.4900 - acc: 0.3732 - val_loss: 2.3155 - val_acc: 0.3992
Epoch 90/100
 - 18s - loss: 2.4891 - acc: 0.3720 - val_loss: 2.2573 - val_acc: 0.4171
Epoch 91/100
 - 18s - loss: 2.4869 - acc: 0.3680 - val_loss: 2.3686 - val_acc: 0.3924
Epoch 92/100
 - 18s - loss: 2.4923 - acc: 0.3690 - val_loss: 2.3335 - val_acc: 0.4069
Epoch 93/100
 - 18s - loss: 2.4844 - acc: 0.3714 - val_loss: 2.2983 - val_acc: 0.4053
Epoch 94/100
 - 19s - loss: 2.4796 - acc: 0.3707 - val_loss: 2.2780 - val_acc: 0.4160
Epoch 95/100
 - 18s - loss: 2.4885 - acc: 0.3705 - val_loss: 2.3367 - val_acc: 0.3975
Epoch 96/100
 - 18s - loss: 2.4938 - acc: 0.3691 - val

Epoch 81/100
 - 18s - loss: 2.4677 - acc: 0.3714 - val_loss: 2.2694 - val_acc: 0.4140
Epoch 82/100
 - 18s - loss: 2.4631 - acc: 0.3739 - val_loss: 2.3187 - val_acc: 0.4044
Epoch 83/100
 - 18s - loss: 2.4692 - acc: 0.3725 - val_loss: 2.2453 - val_acc: 0.4141
Epoch 84/100
 - 19s - loss: 2.4712 - acc: 0.3734 - val_loss: 2.2810 - val_acc: 0.4117
Epoch 85/100
 - 19s - loss: 2.4683 - acc: 0.3728 - val_loss: 2.2245 - val_acc: 0.4145
Epoch 86/100
 - 18s - loss: 2.4722 - acc: 0.3729 - val_loss: 2.1705 - val_acc: 0.4330
Epoch 87/100
 - 18s - loss: 2.4706 - acc: 0.3740 - val_loss: 2.2264 - val_acc: 0.4276
Epoch 88/100
 - 19s - loss: 2.4685 - acc: 0.3713 - val_loss: 2.2721 - val_acc: 0.4090
Epoch 89/100
 - 18s - loss: 2.4636 - acc: 0.3745 - val_loss: 2.3480 - val_acc: 0.3910
Epoch 90/100
 - 18s - loss: 2.4692 - acc: 0.3746 - val_loss: 2.3208 - val_acc: 0.4201
Epoch 91/100
 - 18s - loss: 2.4762 - acc: 0.3734 - val_loss: 2.2210 - val_acc: 0.4259
Epoch 92/100
 - 19s - loss: 2.4739 - acc: 0.3741 - val

Epoch 77/100
 - 20s - loss: 2.4792 - acc: 0.3735 - val_loss: 2.1842 - val_acc: 0.4348
Epoch 78/100
 - 20s - loss: 2.4775 - acc: 0.3739 - val_loss: 2.2237 - val_acc: 0.4295
Epoch 79/100
 - 20s - loss: 2.4749 - acc: 0.3720 - val_loss: 2.3398 - val_acc: 0.4183
Epoch 80/100
 - 20s - loss: 2.4792 - acc: 0.3700 - val_loss: 2.2153 - val_acc: 0.4243
Epoch 81/100
 - 20s - loss: 2.4752 - acc: 0.3726 - val_loss: 2.1962 - val_acc: 0.4401
Epoch 82/100
 - 19s - loss: 2.4727 - acc: 0.3727 - val_loss: 2.2184 - val_acc: 0.4384
Epoch 83/100
 - 18s - loss: 2.4710 - acc: 0.3726 - val_loss: 2.3126 - val_acc: 0.4186
Epoch 84/100
 - 18s - loss: 2.4707 - acc: 0.3720 - val_loss: 2.2636 - val_acc: 0.4231
Epoch 85/100
 - 18s - loss: 2.4656 - acc: 0.3729 - val_loss: 2.2760 - val_acc: 0.4175
Epoch 86/100
 - 19s - loss: 2.4674 - acc: 0.3741 - val_loss: 2.2211 - val_acc: 0.4263
Epoch 87/100
 - 19s - loss: 2.4701 - acc: 0.3725 - val_loss: 2.2650 - val_acc: 0.4196
Epoch 88/100
 - 19s - loss: 2.4651 - acc: 0.3742 - val

References
- https://github.com/keras-team/keras/tree/master/examples
- https://keras.io/visualization/